In [5]:
import pandas as pd
import os

# main_dir = 
Dataset_name = 'TechLabsDataset.csv' 
DataDictionary_name = 'TechLabsDataset_Dictionary.csv'

Dataset = pd.read_csv(os.path.join('./Data/',Dataset_name), index_col = 0)
DataDictionary = pd.read_csv(os.path.join('./Data/',DataDictionary_name), index_col = 0)



,name,type_company,lat,lon,city,avg_stars_num,n_reviews_num,encoded_user,local_guide,clean_other_review_num,published_date,today_date,stars_num,review_EN,original_lang
0,Caretrex warehousing & logistics,Logistics service,51.593721,5.073492,Tilburg,5.0,2.0,HHpDM,0.0,4.0,2 years ago,2022-09-24,5.0,NaN,NaN
1,Caretrex warehousing & logistics,Logistics service,51.593721,5.073492,Tilburg,5.0,2.0,dYFWx,0.0,1.0,5 years ago,2022-09-24,5.0,NaN,NaN
2,FEFA Logistics,Trucking company,51.480074,5.446764,Eindhoven,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dekkers Transport Holland,Trucking company,51.597249,5.027990,Tilburg,4.6,53.0,Pp1)c,0.0,5.0,a year ago,2022-09-24,5.0,Best employer ever. Worked there for approxima...,ENG
4,Dekkers Transport Holland,Trucking company,51.597249,5.027990,Tilburg,4.6,53.0,*nk1l,0.0,2.0,a year ago,2022-09-24,5.0,better place with very good people everything ...,ENG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8075,Van Thiel Logistics,Trucking company,51.827945,5.765815,Nijmegen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8076,InTime Logistics & Consulting,Trucking company,52.049578,5.117079,Nieuwegein,4.7,3.0,wqc@z,1.0,13.0,9 months ago,2022-09-22,5.0,Unique\n\n,NOT_ENG
8077,InTime Logistics & Consulting,Trucking company,52.049578,5.117079,Nieuwegein,4.7,3.0,0*Rv2,0.0,1.0,8 months ago,2022-09-22,5.0,NaN,NaN
8078,InTime Logistics & Consulting,Trucking company,52.049578,5.117079,Nieuwegein,4.7,3.0,7GAL*,0.0,2.0,2 years ago,2022-09-22,4.0,NaN,NaN


### Scraping test

1. Get URLs for all companies
2. Visit URLs using scraper (beautifulsoup + requests)
3. Count occurences of a list of keywords ("sustainability", etc.)
4. Scrape all internal links (recursively) and do the same, accumulating count. Don't scrape the pages you already did.
5. Store number of keyword mentions per company in dataframe or csv

In [2]:
from googlesearch import search

unique_companies = pd.DataFrame(Dataset).drop_duplicates(subset=["name"])

urls = []

for company in unique_companies["name"].head(10):
    url = next(search(company))
    urls.append((company, url))



In [3]:
print(urls)

[('Caretrex warehousing & logistics', 'http://www.caretrex.com/'), ('FEFA Logistics', 'https://www.facebook.com/fefalogistics/'), ('Dekkers Transport Holland', 'https://dekkerstransport.nl/en/'), ('Cotra Logistics Solutions BV', 'https://www.cotra.eu/'), ('Leeuw Shipping & Logistics G de', 'http://www.tupalo.net/en/ridderkerk/leeuw-shipping-and-logistics-g-de'), ('Dedicated Logistics Services', 'http://www.dlstrans.com/'), ('Schenker Logistics Nederland', 'https://www.dbschenker.com/nl-en'), ('Milloa Logistics', 'https://www.emis.com/php/company-profile/PL/Milla_Logistics_Sp_z_oo_en_12711249.html'), ('Bos Logistics Schiphol', 'https://www.dnb.com/business-directory/company-profiles.bos_logistics_bv.fce49e84f77986c3a4a0a1e940a5e5ba.html'), ('Allroad Logistics', 'https://www.allroadlogistica.com/usa/')]


In [4]:
import requests
from bs4 import BeautifulSoup
import re

links_to_visit = []

# EXAMPLE
def crawl_page(url):
    try:
        # Get page content
        page = requests.get(url, timeout=5)

        # Count occurences of a set of words (sustainable, etc.)
        word_count = page.text.count("sustainable")
        print(f"Sustainable count: {word_count}")

        soup = BeautifulSoup(page.content, "html.parser")

        # Get internal links on this page
        linkTags = soup.find_all(name="a")
        links = [x.attrs.get("href") for x in linkTags]
        internal_links = [x for x in links if x != None and (x.startswith("/") or x.startswith(url))]
        print(f"Internal link count: {len(internal_links)}")
        # Filter links to only contain internal links
        # ...
        # links_to_visit.append(links)
    except Exception:
        print(f"Failed with {url}")


for company, url in urls:
    print(f"Crawling {company} at {url}")

    crawl_page(url)


Crawling Caretrex warehousing & logistics at http://www.caretrex.com/
Sustainable count: 0
Internal link count: 9
Crawling FEFA Logistics at https://www.facebook.com/fefalogistics/
Sustainable count: 0
Internal link count: 0
Crawling Dekkers Transport Holland at https://dekkerstransport.nl/en/
Sustainable count: 0
Internal link count: 16
Crawling Cotra Logistics Solutions BV at https://www.cotra.eu/
Sustainable count: 0
Internal link count: 26
Crawling Leeuw Shipping & Logistics G de at http://www.tupalo.net/en/ridderkerk/leeuw-shipping-and-logistics-g-de
Sustainable count: 0
Internal link count: 2
Crawling Dedicated Logistics Services at http://www.dlstrans.com/
Sustainable count: 0
Internal link count: 9
Crawling Schenker Logistics Nederland at https://www.dbschenker.com/nl-en
Sustainable count: 0
Internal link count: 161
Crawling Milloa Logistics at https://www.emis.com/php/company-profile/PL/Milla_Logistics_Sp_z_oo_en_12711249.html
Sustainable count: 0
Internal link count: 210
Craw